In [4]:
%cd ..

/home/haito/kaggle/rsna-str/workdir


In [5]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append(".")
from src.factory import *
from src.utils import *
from sklearn.metrics import log_loss

DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")


In [47]:
train = pd.read_csv(DATADIR / "train.csv")

pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

portion = pd.read_csv(DATADIR / "study_pos_portion.csv")
train = train.merge(portion, on="StudyInstanceUID")

z_pos_df = pd.read_csv(DATADIR / "sop_to_prefix.csv").rename(columns={'img_prefix': 'z_pos'})
train = train.merge(z_pos_df, on="SOPInstanceUID")

train = train.query("fold == 0 or fold == 1")  # now I have fold0,1 only
studies = train.StudyInstanceUID.unique()

# agg = t.groupby("StudyInstanceUID")["SOPInstanceUID"].apply(list)
# agg_one = t.groupby("StudyInstanceUID").first()
# t = t.set_index("SOPInstanceUID")

In [48]:
def get_pred(_path):
    res = load_pickle(_path)
    raw_pred = pd.DataFrame({
        "SOPInstanceUID": res["ids"],
        **res["outputs"],
    })
    return raw_pred
    # return raw_pred.set_index("sop")

def calib_p(arr, factor):  # set factor>1 to enhance positive prob
    return arr * factor / (arr * factor + (1-arr))

In [49]:
valid_fold0 = "output/035_pe_present___448/valid.fold0-ep1.picle"
valid_fold0_calib_f = 8.555037588568537
valid_fold1 = "output/035_pe_present___448___apex___resume/valid.fold1-ep1.pickle"
valid_fold1_calib_f = 5.72045

oof_f0 = get_pred(valid_fold0)
oof_f1 = get_pred(valid_fold1)

if True: ## do calib for each fold
    oof_f0["pe_present_on_image"] = calib_p(oof_f0["pe_present_on_image"], valid_fold0_calib_f)
    oof_f1["pe_present_on_image"] = calib_p(oof_f1["pe_present_on_image"], valid_fold1_calib_f)

oof = pd.concat([oof_f0, oof_f1]).rename(columns={'pe_present_on_image': 'pred'})

train = train.merge(oof[['pred', 'SOPInstanceUID']], on="SOPInstanceUID")  # add pred

In [54]:
train_copyed = train.copy()
train.columns

Index(['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID',
       'pe_present_on_image', 'negative_exam_for_pe', 'qa_motion',
       'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
       'leftsided_pe', 'chronic_pe', 'true_filling_defect_not_pe',
       'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate',
       'exam_type', 'fold', 'pe_present_portion', 'z_pos', 'pred'],
      dtype='object')

In [ ]:
""" feature engineer """

In [55]:
# train = pd.read_pickle('../input/train_with_position.pkl')
train = train.sort_values(['StudyInstanceUID', 'z_pos'])
# train = train.merge(oof[['pred', 'SOPInstanceUID']], on='SOPInstanceUID')

# test = pd.read_pickle('../input/test_with_position.pkl')
# test = test.sort_values(['StudyInstanceUID', 'z_pos'])
# test = test.merge(test_pred[pe_present_portion], on='SOPInstanceUID')

# train = train.merge(oof_non_weight[['pred_non_weight', 'SOPInstanceUID']], on='SOPInstanceUID')
# test = test.merge(test_pred_non_weight[['pred_non_weight', 'SOPInstanceUID']], on='SOPInstanceUID')
# train['pred_mean'] = (train['pred'] + train['pred_non_weight']) / 2
# test['pred_mean'] = (test['pred'] + test['pred_non_weight']) / 2

train_current_z_pos = train.groupby('StudyInstanceUID')['z_pos'].shift(0)
# test_current_z_pos = test.groupby('StudyInstanceUID')['z_pos'].shift(0)
for i in range(1, 20):
    # train[f'pre_mean{i}'] = train.groupby('StudyInstanceUID')['pred_mean'].shift(i)
    # train[f'post_mean{i}'] = train.groupby('StudyInstanceUID')['pred_mean'].shift(-i)
    train[f'pre{i}'] = train.groupby('StudyInstanceUID')['pred'].shift(i)
    train[f'post{i}'] = train.groupby('StudyInstanceUID')['pred'].shift(-i)
    # train[f'pre_non_weight{i}'] = train.groupby('StudyInstanceUID')['pred_non_weight'].shift(i)
    # train[f'post_non_weight{i}'] = train.groupby('StudyInstanceUID')['pred_non_weight'].shift(-i)
    
    # test[f'pre_mean{i}'] = test.groupby('StudyInstanceUID')['pred_mean'].shift(i)
    # test[f'post_mean{i}'] = test.groupby('StudyInstanceUID')['pred_mean'].shift(-i)
    # test[f'pre{i}'] = test.groupby('StudyInstanceUID')['pred'].shift(i)
    # test[f'post{i}'] = test.groupby('StudyInstanceUID')['pred'].shift(-i)
    # test[f'pre_non_weight{i}'] = test.groupby('StudyInstanceUID')['pred_non_weight'].shift(i)
    # test[f'post_non_weight{i}'] = test.groupby('StudyInstanceUID')['pred_non_weight'].shift(-i)

for i in [1]:
    train[f'pre_z_pos_diff{i}'] = train_current_z_pos - train.groupby('StudyInstanceUID')['pred'].shift(i)
    # test[f'pre_z_pos_diff{i}'] = test_current_z_pos - test.groupby('StudyInstanceUID')['pred'].shift(i)



In [59]:
train

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,...,post15,pre16,post16,pre17,post17,pre18,post18,pre19,post19,pre_z_pos_diff1
558384,0003b3d648eb,d2b2960c2bbf,14605bcc564c,0,1,0,0,0,0,0,...,0.000145,NaN,0.000095,NaN,0.000078,NaN,0.000102,NaN,0.000166,NaN
558367,0003b3d648eb,d2b2960c2bbf,d0849d3b6507,0,1,0,0,0,0,0,...,0.000095,NaN,0.000078,NaN,0.000102,NaN,0.000166,NaN,0.000127,0.999928
558324,0003b3d648eb,d2b2960c2bbf,18928b724b69,0,1,0,0,0,0,0,...,0.000078,NaN,0.000102,NaN,0.000166,NaN,0.000127,NaN,0.000175,1.999923
558359,0003b3d648eb,d2b2960c2bbf,56bc011203f4,0,1,0,0,0,0,0,...,0.000102,NaN,0.000166,NaN,0.000127,NaN,0.000175,NaN,0.000263,2.999976
558312,0003b3d648eb,d2b2960c2bbf,24b933c4c518,0,1,0,0,0,0,0,...,0.000166,NaN,0.000127,NaN,0.000175,NaN,0.000263,NaN,0.000334,3.999920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98085,fffda3f22362,39ca5eaafffe,29e855db7f2b,0,1,0,0,0,0,0,...,NaN,0.002196,NaN,0.002147,NaN,0.002662,NaN,0.002301,NaN,163.998713
98023,fffda3f22362,39ca5eaafffe,f7ca277a66c2,0,1,0,0,0,0,0,...,NaN,0.001723,NaN,0.002196,NaN,0.002147,NaN,0.002662,NaN,164.998323
98131,fffda3f22362,39ca5eaafffe,59714fd8dd25,0,1,0,0,0,0,0,...,NaN,0.001989,NaN,0.001723,NaN,0.002196,NaN,0.002147,NaN,165.997458
98141,fffda3f22362,39ca5eaafffe,b33567349fae,0,1,0,0,0,0,0,...,NaN,0.002083,NaN,0.001989,NaN,0.001723,NaN,0.002196,NaN,166.996255


In [64]:
ids = [c for c in list(train) if 'UID' in c]
targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
]
other_targets = [c for c in list(train) if 'pe_present_on_image' in c]
### remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + targets + ids + other_targets
remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + ['exam_type','flow_artifact','pe_present_portion', 'true_filling_defect_not_pe'] + targets + ids + other_targets

features = sorted(list(set(list(train)) - set(remove_cols)))
print(features)

['post1', 'post10', 'post11', 'post12', 'post13', 'post14', 'post15', 'post16', 'post17', 'post18', 'post19', 'post2', 'post3', 'post4', 'post5', 'post6', 'post7', 'post8', 'post9', 'pre1', 'pre10', 'pre11', 'pre12', 'pre13', 'pre14', 'pre15', 'pre16', 'pre17', 'pre18', 'pre19', 'pre2', 'pre3', 'pre4', 'pre5', 'pre6', 'pre7', 'pre8', 'pre9', 'pre_z_pos_diff1', 'pred', 'z_pos']


In [108]:
features_copyed = features.copy()

In [66]:
def fobj(pred, data):
    true = data.get_label()
    label = 2*true - 1
    weights = data.weights
    response = -label / (1 + np.exp(label * pred))
    abs_response = np.abs(response)
    grad = response
    hess = abs_response * (1 - abs_response)
    return grad*weights, hess*weights

In [97]:
import torch
bce_func = torch.nn.BCELoss(reduction='none')

def feval2(preds, data):
    scores = bce_func(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [102]:
#### 推測して作成したもの. Yujiに確認する必要がある
import torch
bce_func_logit = torch.nn.BCEWithLogitsLoss(reduction='none')

def feval(preds, data):
    scores = bce_func_logit(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [105]:
import torch
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [118]:
features = ['pred']
features = features_copyed

In [119]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
import warnings
warnings.simplefilter('ignore')
import pickle

oof_preds_list = []
test_preds_list = []
models_list = []
target = 'pe_present_on_image'

for i in range(1):
    print(f'=================={i}================')
    if i % 4 == 0:
        params = {'boosting_type': 'gbdt',
            'objective': 'binary',
#             'metric': 'None',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100,
            } 
    elif i % 4 == 1:
        params = {
            'max_depth': 4,
            'max_leave': int(0.2 * 2 ** 4),
            'reg_lambda': 1,
            'reg_alpha': 1,
            'subsamples': 0.8,
            'colsample_bytree': 0.7,
            'objective': 'binary',
            'min_data_in_leaf': 0,
            'boosting': 'gbdt',
            'metric': 'None',
            'learning_rate': 0.1,
                      }
    elif i % 4 == 2:
        params = {
            'num_leaves': 19, 
            'min_data_in_leaf': 160,
            'min_child_weight': 0.03,
            'bagging_fraction' : 0.7,
            'feature_fraction' : 0.8,
            'learning_rate' : 0.1,
            'max_depth': -1,
            'reg_alpha': 0.02,
            'reg_lambda': 0.12,
            'objective': 'binary',
            'verbose': 100,
            'boost_from_average': False,
            'metric': 'None',
        }  
    else:
        params = {
            'objective': "binary",
            'metric': 'None',
            'boost_from_average': "false",
            'tree_learner': "serial",
            'max_depth': -1,
            'learning_rate': 0.1,
            'num_leaves': 197,
            'feature_fraction': 0.3,
            'bagging_freq': 1,
            'bagging_fraction': 0.7,
            'min_data_in_leaf': 100,
            'bagging_seed': 11,
            'max_bin': 255,
            'verbosity': -1}    
        
    oof_preds = np.zeros(train.shape[0])
    ### test_preds = np.zeros(test.shape[0])
    val_results = {}
    models = []
    params['random_state'] = i
    iter = 100000
#     for n_fold, (trn_idx, val_idx) in enumerate(kf.split(train, train[target])):

    ### for n_fold in range(5):
    for n_fold in range(2):
        tr = train.query(f'fold != {n_fold}')
        val = train.query(f'fold == {n_fold}')
        trn_data = lgb.Dataset(tr[features], label=tr[target])
        trn_data.weights = tr.pe_present_portion.values
        val_data = lgb.Dataset(val[features], label=val[target])
        val_data.weights = val.pe_present_portion.values
        
        clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
#                         verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
                        feval=feval, fobj = fobj, verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
        file = f'lgbs/lgb_seed{i}_fold{n_fold}.pkl'
        pickle.dump(clf, open(file, 'wb'))
#         models.append(clf)
        oof_preds[train.fold==n_fold] = clf.predict(val[features])
        ### test_preds += clf.predict(test[features]) / 5
#     models_list.append(models)
    oof_preds_list.append(oof_preds)
    ### test_preds_list.append(test_preds)

print(f'-------------------------------------------------------------------------roc_auc: {roc_auc_score(train[target], np.mean(oof_preds_list, axis=0))}')
print(f'----------------------------------------------------------roc_auc using raw pred: {roc_auc_score(train[target], train["pred"])}')
print(f'------------------------------------------------------------------------------AP: {average_precision_score(train[target], np.mean(oof_preds_list, axis=0))}')
print(f'---------------------------------------------------------------AP using raw pred: {average_precision_score(train[target], train["pred"])}')

lgb_oof = np.mean(oof_preds_list, axis=0)
lgb_preds = np.mean(test_preds_list, axis=0)
train['lgb_preds'] = sigmoid(lgb_oof)
### test['lgb_preds'] = sigmoid(lgb_preds)

ata to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Re-bagging, using 270700 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 8
[LightGBM] [Debug] Re-bagging, using 270502 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 7
[LightGBM] [Debug] Re-bagging, using 270551 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 7
[LightGBM] [Debug] Re-bagging, using 270633 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 7
[LightGBM] [Debug] Re-bagging, using 270652 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 10
[LightGBM] [Debug] Re-bagging, using 270746 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 7
[LightGBM] [Debug] Re-bagging, using 270784 data to train
[LightGBM] [Debug] Trained a tree with leaves = 31 and max_depth = 11
[LightGBM] [Debug] Re-baggi

In [ ]:
import torch
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

bce_func = torch.nn.BCELoss(reduction='none')

lgb_losses = bce_func(torch.FloatTensor(sigmoid(lgb_oof)), torch.FloatTensor(train['pe_present_on_image']))

torch.mean(lgb_losses*train['weight'].values)